So far, this is just a proof of concept for using the API. Obviously, we need to streamline the process of pulling more rows, loading/saving the csv as well as cleaning the data. 

In [90]:
%pip install requests keyring

Note: you may need to restart the kernel to use updated packages.


In [91]:
from cdg_client import CDGClient
import xml.etree.ElementTree as ET
import xmltodict
import pandas as pd

In [92]:
API_KEY_PATH = '../../../api_key.txt'       # put your api_key path here
                                            # it shouldn't go to github
LIMIT = 10 # testing with a small number since amount per day is capped 
parse_xml = lambda data: ET.fromstring(data)


with open(API_KEY_PATH) as file:
    api_key = file.readline()
client = CDGClient(api_key, response_format='xml')
data, _ = client.get(f'bill?format=xml&limit={LIMIT}')

In [93]:
bills = xmltodict.parse(data)['api-root']['bills']['bill']
df = pd.DataFrame(bills)
df.head()

,congress,type,originChamber,originChamberCode,number,url,title,updateDateIncludingText,latestAction,updateDate
0,119,SRES,Senate,S,420,https://api.congress.gov/v3/bill/119/sres/420?...,A resolution supporting the designation of Sep...,2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Submitte...",2025-09-30
1,119,SRES,Senate,S,418,https://api.congress.gov/v3/bill/119/sres/418?...,A resolution expressing support for the design...,2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Resoluti...",2025-09-30
2,119,S,Senate,S,2144,https://api.congress.gov/v3/bill/119/s/2144?fo...,A bill to improve the safety and security of M...,2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Passed S...",2025-09-30
3,119,S,Senate,S,1333,https://api.congress.gov/v3/bill/119/s/1333?fo...,Strengthening Child Exploitation Enforcement Act,2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Passed S...",2025-09-30
4,119,HR,House,H,5371,https://api.congress.gov/v3/bill/119/hr/5371?f...,"Continuing Appropriations and Extensions Act, ...",2025-09-30,"{'actionDate': '2025-09-29', 'text': 'Cloture ...",2025-09-30


In [94]:
# get the next LIMIT # of bills and add it to the data frame over and over
for i in range(1, 5):
    offset = i * LIMIT
    more_data, _ = client.get(f'bill?format=json&limit={LIMIT}&offset={offset}') 
    more_bills = xmltodict.parse(more_data)['api-root']['bills']['bill']
    more_rows = pd.DataFrame(more_bills)
    df = pd.concat([df, more_rows], ignore_index=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   congress                 50 non-null     object
 1   type                     50 non-null     object
 2   originChamber            50 non-null     object
 3   originChamberCode        50 non-null     object
 4   number                   50 non-null     object
 5   url                      50 non-null     object
 6   title                    50 non-null     object
 7   updateDateIncludingText  50 non-null     object
 8   latestAction             50 non-null     object
 9   updateDate               50 non-null     object
dtypes: object(10)
memory usage: 4.0+ KB


In [95]:
# save as csv 
# # probably want to go to dropbox or something, not directly in the repository
df.to_csv('congress_bills.csv') 